# Initialization

In [ ]:
# using Pkg
# Pkg.add(["Plots", "WAV", "Wavelets", "Linear Algebra"])

In [23]:
using Plots, WAV, FFTW, Wavelets, LinearAlgebra
include("./wavplay-pulse.jl")

# enable plotly backend for interactive plots
plotly(size=(800,300), lw = 2, lc = :black)

Plots.PlotlyBackend()

# What is a Wavelet

In order to understand what a wavelet is, it is recommened to have some experience with the Fourier Transform. A wavelet is a wave oscillation localized in time, this means that is it finite instead of the sine and cosine waves of the Fourier Transform that are infinate. You can see an example in the following plot.

![](./wavelet1.jpg)

In [ ]:
WT.Morlet.(-4, 4, 1:1000 |> collect) |> plot

The word _wavelet_ is derived from the french word _ondelette_, meaning _small wave_ and that is exactly what a wavelet it. But how can it be used to extract the underlying frequency components of a wave ? In the Fourier Transform a signal $g(t)$ is compared against sine and cosine waves of different frequencies, this is not far from how the Wavelet Transform works. Instead of trying different frequencies, the Wavelet Transform alters two wavelet parameters called _scale_ and _shift_. The _scale_ parameter affects the length of the wave and consequently the frequency that it maatches. A larger scale means a lower frequency and a smaller scale translates to a higher frequency. The _shift_ parameter affects the wavelet's position in the time dimension, a lower or higher shift means that the wave will occur sooner or later respectively.

## Families of Wavelets

TODO

# The General Wavelet Transform Equation

$$\LARGE X_{a,b} = \int_{-\infty}^{\infty} \! x(t) * \psi_{a,b}(t) \, \mathrm{d}t $$

TODO

## Continuous and Discrete Wavelet Transforms

TODO

# Why use the Wavelet Transform ?

One of the most important advantages of the Wavelet Transform is that it keep both frequency and time information. In contrast, the Fourier Transform can represent a signal in the frequency domain but not in the time domain. This deficiency can be especially important when analyzing non-stationary signals in which frequency changes over time.  

In the following cells we showcase this ues case, firstly we define some helper functions for creating a non-stationary signal and transforming it to the frequency domain. Then we generate signals and compare the output of the fourier transform of a stationary and a non-stationary signal.

In [7]:
function sin_wave(frequency,amplitude,fs,τ)
    #=
    frequency: cycles per second
    amplitude: multiplier to the sin
    fs:        samples per second
    τ:         total time duration of the wave
    =#
    xs = range(0, τ, length=τ*fs)
    amplitude * sin.(2pi * frequency * xs) # last value is returned automatically
end

multi_sin_wave(freqs, ampl, fs, τ) = sin_wave.(freqs, ampl, fs, τ) |> (x->vcat(x...))

function frequency_domain(y, fs)
    y_fft = fft(y) .|> abs
    y_fft = y_fft[1:Int64(length(y) / 2) + 1] / length(y)
    y_fft[2:end-1] *= 2
    freqs = range(0, fs / 2, step = fs / length(y))
    (freqs, y_fft)
end

mult_sin_wave (generic function with 1 method)

In [9]:
fs = 100
y_cont = multi_sin_wave([2,2,2,2], [1,1,1,1], fs, [2,2,2,2])
y_var = multi_sin_wave([1,2,3,4], [1,1,1,1], fs, [2,2,2,2])

plot([y_cont, y_var], layout = (2,1), lab = ["Continuous Frequency" "Variable Frequency"], lc = [:blue :green])

<!DOCTYPE html>
 
 
 Plots.jl

In [32]:
fft_cont, fft_var = frequency_domain.([y_cont, y_var], fs)
plot([fft_cont[1], fft_var[1]], [fft_cont[2], fft_var[2]], layout = (2,1), xlabel = "Frequency", ylabel = "Amplitude", lc = [:blue :green], lab = ["Continuous Frequency" "Variable Frequency"])

<!DOCTYPE html>
 
 
 Plots.jl

It is obvious from the above plot that the Fourier Transform falls short on non-stationary signals even in this low complexity case where the signal does not vary much frequency-wise.

In [39]:
# add scaleogram
let y = y_var, wt = WT.morl
    yt = cwt(y, wavelet(wt))
    yt = reshape(yt, size(yt, 1), size(yt, 2))
    hm = heatmap(real.(yt)', xlabel="Time", ylabel="Frequency", colorbar=false, color=:magma)
    p = plot(y)
    plot(p, hm, layout = (2,1))
end

<!DOCTYPE html>
 
 
 Plots.jl

It is important to note that reseachers have developed the Short Time Fourier Transform (STFT) to make the Fourier Transform viable in the aforementioned scenario. This method splits the signal into smaller sub-signals using a window size and then performs Fourier Transform on these windows. However problems arise when the window size is too small or too large. In the first case frequency accuracy takes a hit because in the Fourier Transform, the longer a frequency repeats the stronger the accuracy gets. In the second case, time information is lost resulting in less localized and more crude results.

# Application: Signal Denoising

In [ ]:
y, fs, _, _ = wavread("joe.wav");

In [ ]:
yt = cwt(y, wavelet(WT.morl))
yt = reshape(yt, size(yt, 1), size(yt, 2))

In [ ]:
ht = heatmap(real.(yt)', xlabel="Time", ylabel="Frequency", colorbar=true, color=:magma)
p = plot(y)
plot(p, ht, layout = (2,1))